In [ ]:
# import sys
# !python -m pip install pyspark==3.2.1



In [ ]:
# !python -m pip install findspark

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:

scala_version = '2.12'
spark_version = '3.2.1'
packages = [ f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}' , 'org.apache.kafka:kafka-clients:3.3.1' ]
spark = SparkSession.builder.master("local").appName("kafka-example").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

In [3]:
#create dataframe from Kafka data
topic_name='RandomNumber'
kafka_server= 'localhost:9092'
kafkaDf=spark.read.format("kafka").option("kafka.bootstrap.servers",kafka_server)\
    .option("subscribe",topic_name).option("startingOffsets","earliest").load()

In [4]:
#show data
kafkaDf.toPandas()

,key,value,topic,partition,offset,timestamp,timestampType
0,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,0,2022-10-28 21:12:58.414,0
1,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,1,2022-10-28 21:13:03.772,0
2,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,2,2022-10-28 21:13:08.781,0
3,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,3,2022-10-28 21:13:13.793,0
4,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,4,2022-10-28 21:13:18.798,0
...,...,...,...,...,...,...,...
107,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,107,2022-10-28 21:53:47.435,0
108,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,108,2022-10-28 21:53:52.446,0
109,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,109,2022-10-28 21:53:57.455,0
110,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,110,2022-10-28 21:54:02.464,0


In [6]:
len(kafkaDf.toPandas())

112

In [5]:
from dis import dis
from time import sleep
from traceback import print_tb
from IPython.display import display,clear_output

In [6]:
#show streaming data using for loop
batchDF=kafkaDf.select(col('topic'),col('offset'),col('value').cast('string').substr(12,1).alias('rand_number'))

In [7]:
display(batchDF.toPandas())

,topic,offset,rand_number
0,RandomNumber,0,0
1,RandomNumber,1,1
2,RandomNumber,2,2
3,RandomNumber,3,3
4,RandomNumber,4,4
...,...,...,...
107,RandomNumber,107,3
108,RandomNumber,108,3
109,RandomNumber,109,3
110,RandomNumber,110,3


In [8]:

for x in range(0,2000):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Sencond passed: {x*5}")
        display(batchDF.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view ends ...")

Showing live view refreshed every 5 seconds
Sencond passed: 5


,topic,offset,rand_number
0,RandomNumber,0,0
1,RandomNumber,1,1
2,RandomNumber,2,2
3,RandomNumber,3,3
4,RandomNumber,4,4
...,...,...,...
107,RandomNumber,107,3
108,RandomNumber,108,3
109,RandomNumber,109,3
110,RandomNumber,110,3


break
Live view ends ...


Perform some data aggreation and show live results

In [9]:
batchCountDF = batchDF.groupBy('rand_number').count()
for x in range (0,2000):
    try:
        print('showing live view refreshed every 5 seconds')
        print(f"Second passed: {x*5}")
        display(batchCountDF.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view ended")

showing live view refreshed every 5 seconds
Second passed: 5


,rand_number,count
0,7,3
1,3,20
2,8,3
3,0,5
4,5,13
5,6,3
6,9,3
7,1,25
8,4,13
9,2,24


break
Live view ended


Create Streaming dataframe from Kafka

In [17]:
#startingOffset: earliest for batch  and lastest for streaming
streamRawDf = spark.readStream.format("kafka").option("kafka.bootstrap.servers", kafka_server).option("subscribe", topic_name).option("startingOffsets","lastest").load()
streamDF = streamRawDf.select(col('topic'),col('offset'),col('value').cast('string').substr(12,1).alias('rand_number'))
checkEvenDF = streamDF.withColumn('Is_Even',col('rand_number').cast('int') % 2 == 0 )

Write stream

In [18]:
from random import randint
randNum=str(randint(0,10000))
q1name = "queryNumber"+randNum
q2name = "queryCheckEven"+randNum

print(q1name+q2name)

queryNumber692queryCheckEven692


In [19]:
stream_writer1 = (streamDF.writeStream.queryName(q1name).trigger(processingTime="5 seconds").outputMode("append").format("memory"))
stream_writer2 = (checkEvenDF.writeStream.queryName(q2name).trigger(processingTime="5 seconds").outputMode("append").format("memory"))

In [20]:
query1 = stream_writer1.start()
query1.awaitTermination(1)

False

In [21]:
query1.explain()

== Physical Plan ==
WriteToDataSourceV2 org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@67f24e5, org.apache.spark.sql.execution.datasources.v2.DataSourceV2Strategy$$Lambda$3761/0x00000008020425c0@3c17e17b
+- *(1) Project [topic#184, offset#186L, substring(cast(value#183 as string), 12, 1) AS rand_number#196]
   +- MicroBatchScan[key#182, value#183, topic#184, partition#185, offset#186L, timestamp#187, timestampType#188] class org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaScan




In [22]:
query2 = stream_writer2.start()

query2.awaitTermination(1)

False

In [15]:
print(query1.name+" "+query2.name)

queryNumber1116 queryCheckEven1116


In [23]:
spark.sql(f"select * from {q1name}").show()

+------------+------+-----------+
|       topic|offset|rand_number|
+------------+------+-----------+
|RandomNumber|     0|          0|
|RandomNumber|     1|          1|
|RandomNumber|     2|          2|
|RandomNumber|     3|          3|
|RandomNumber|     4|          4|
|RandomNumber|     5|          5|
|RandomNumber|     6|          6|
|RandomNumber|     7|          7|
|RandomNumber|     8|          8|
|RandomNumber|     9|          9|
|RandomNumber|    10|          0|
|RandomNumber|    11|          1|
|RandomNumber|    12|          2|
|RandomNumber|    13|          3|
|RandomNumber|    14|          4|
|RandomNumber|    15|          5|
|RandomNumber|    16|          6|
|RandomNumber|    17|          7|
|RandomNumber|    18|          8|
|RandomNumber|    19|          9|
+------------+------+-----------+
only showing top 20 rows



View streaming result

In [24]:
result1=spark.sql(f"SELECT * from {query1.name}")
result1.toPandas()

,topic,offset,rand_number
0,RandomNumber,0,0
1,RandomNumber,1,1
2,RandomNumber,2,2
3,RandomNumber,3,3
4,RandomNumber,4,4
...,...,...,...
107,RandomNumber,107,3
108,RandomNumber,108,3
109,RandomNumber,109,3
110,RandomNumber,110,3


In [31]:
# query1.

'fb162849-5302-46a6-a0af-486511d5c64b'

In [25]:
for x in range(0,2000):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"second passed: {x*5}")
        result1=spark.sql(f"SELECT * from {query1.name}")
        result2=spark.sql(f"SELECT * from {query2.name}")
        display(result1.toPandas())
        display(result2.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view end")


Showing live view refreshed every 5 seconds
second passed: 5


,topic,offset,rand_number
0,RandomNumber,0,0
1,RandomNumber,1,1
2,RandomNumber,2,2
3,RandomNumber,3,3
4,RandomNumber,4,4
...,...,...,...
107,RandomNumber,107,3
108,RandomNumber,108,3
109,RandomNumber,109,3
110,RandomNumber,110,3


,topic,offset,rand_number,Is_Even
0,RandomNumber,0,0,True
1,RandomNumber,1,1,False
2,RandomNumber,2,2,True
3,RandomNumber,3,3,False
4,RandomNumber,4,4,True
...,...,...,...,...
107,RandomNumber,107,3,False
108,RandomNumber,108,3,False
109,RandomNumber,109,3,False
110,RandomNumber,110,3,False


break
Live view end
